# Modelo de Classificação

Este modelo tem como  objetivo prever se o pagamento de uma nota fiscal será feito no prazo ou com atraso, utilizando como base características da própria nota e informações adicionais.

O modelo utiliza um classificador Random Forest, que é uma técnica de aprendizado supervisionado amplamente usada para problemas de classificação binária.

Preparação dos dados: Inclui a transformação de datas e variáveis categóricas em formato numérico, como o tempo entre a data de criação e a data de vencimento da nota, e a aplicação de One-Hot Encoding para variáveis categóricas como o tipo de serviço e o estado da nota.

Criação da variável alvo: A variável-alvo é definida com base no status de vencimento da nota, onde 1 indica que a nota está vencida e 0 que está no prazo.

Treinamento do modelo: O modelo foi treinado utilizando 80% dos dados para treino e 20% para teste, tanto em um cenário com quanto sem a variável de localização do pagamento (payment_place), para avaliar seu impacto na previsão.

Avaliação do modelo: O desempenho do modelo foi avaliado utilizando as métricas de acurácia e AUC-ROC, com o objetivo de determinar a eficácia do modelo em prever corretamente o status de vencimento das notas.

In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [64]:
# DataFrame original foi carregado de um CSV
df = pd.read_csv('asset_trade_bills_02.csv', delimiter= ',', low_memory = False) 
df.head()

,id,due_date,nfe_number,nfe_series,kind,state,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,participant_id,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,services,canceled,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12 13:38:12.035807,2024-02-09 12:56:01.418036,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,invoice,123123,São Paulo,others,NaN
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,goods,canceled,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07 19:34:32.482400,2024-02-09 12:56:26.426682,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,BA,others,NaN
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,goods,active,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20 13:22:01.359676,2024-02-01 03:01:47.846091,NaN,ad1b3da5-be27-4282-93df-c3bd77acac91,nfe_key,NaN,RN,NaN,NaN
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,goods,canceled,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29 17:54:29.986491,2023-07-05 20:07:14.565196,NaN,04282b7d-ed52-4bbe-abb8-eea6cc77ba9f,nfe_key,NaN,teste,operational_error,NaN
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,goods,canceled,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07 18:56:32.249501,2024-02-09 12:56:50.403976,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,São Paulo,others,NaN


In [65]:
print(df.isnull().sum())

id                           0
due_date                     0
nfe_number                 980
nfe_series                1198
kind                         0
state                        0
payer_id                     0
endorser_original_id         0
new_due_date                 0
created_at                   0
updated_at                   0
deleted_at              188092
participant_id              11
ballast_kind                 0
invoice_number          189175
payment_place                0
update_reason_kind      147257
finished_at             190218
dtype: int64


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190792 entries, 0 to 190791
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    190792 non-null  object
 1   due_date              190792 non-null  object
 2   nfe_number            189812 non-null  object
 3   nfe_series            189594 non-null  object
 4   kind                  190792 non-null  object
 5   state                 190792 non-null  object
 6   payer_id              190792 non-null  object
 7   endorser_original_id  190792 non-null  object
 8   new_due_date          190792 non-null  object
 9   created_at            190792 non-null  object
 10  updated_at            190792 non-null  object
 11  deleted_at            2700 non-null    object
 12  participant_id        190781 non-null  object
 13  ballast_kind          190792 non-null  object
 14  invoice_number        1617 non-null    object
 15  payment_place    

In [67]:
print('Registros:'+ str(len(df)))

Registros:190792


In [68]:
df['due_date'] = pd.to_datetime(df['due_date'],errors='coerce')

In [69]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

In [70]:
df['created_at'] = df['created_at'].dt.strftime('%Y-%m-%d')

In [71]:
df.head()

,id,due_date,nfe_number,nfe_series,kind,state,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,participant_id,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,services,canceled,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12,2024-02-09 12:56:01.418036,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,invoice,123123,São Paulo,others,NaN
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,goods,canceled,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07,2024-02-09 12:56:26.426682,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,BA,others,NaN
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,goods,active,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20,2024-02-01 03:01:47.846091,NaN,ad1b3da5-be27-4282-93df-c3bd77acac91,nfe_key,NaN,RN,NaN,NaN
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,goods,canceled,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29,2023-07-05 20:07:14.565196,NaN,04282b7d-ed52-4bbe-abb8-eea6cc77ba9f,nfe_key,NaN,teste,operational_error,NaN
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,goods,canceled,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07,2024-02-09 12:56:50.403976,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,São Paulo,others,NaN


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190792 entries, 0 to 190791
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    190792 non-null  object        
 1   due_date              190792 non-null  datetime64[ns]
 2   nfe_number            189812 non-null  object        
 3   nfe_series            189594 non-null  object        
 4   kind                  190792 non-null  object        
 5   state                 190792 non-null  object        
 6   payer_id              190792 non-null  object        
 7   endorser_original_id  190792 non-null  object        
 8   new_due_date          190792 non-null  object        
 9   created_at            190792 non-null  object        
 10  updated_at            190792 non-null  object        
 11  deleted_at            2700 non-null    object        
 12  participant_id        190781 non-null  object        
 13 

In [73]:
# Convertendo para o formato datetime
df['created_at'] = pd.to_datetime(df['created_at'])

In [74]:
df.head()

,id,due_date,nfe_number,nfe_series,kind,state,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,participant_id,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,services,canceled,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12,2024-02-09 12:56:01.418036,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,invoice,123123,São Paulo,others,NaN
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,goods,canceled,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07,2024-02-09 12:56:26.426682,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,BA,others,NaN
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,goods,active,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20,2024-02-01 03:01:47.846091,NaN,ad1b3da5-be27-4282-93df-c3bd77acac91,nfe_key,NaN,RN,NaN,NaN
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,goods,canceled,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29,2023-07-05 20:07:14.565196,NaN,04282b7d-ed52-4bbe-abb8-eea6cc77ba9f,nfe_key,NaN,teste,operational_error,NaN
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,goods,canceled,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07,2024-02-09 12:56:50.403976,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,São Paulo,others,NaN


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190792 entries, 0 to 190791
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    190792 non-null  object        
 1   due_date              190792 non-null  datetime64[ns]
 2   nfe_number            189812 non-null  object        
 3   nfe_series            189594 non-null  object        
 4   kind                  190792 non-null  object        
 5   state                 190792 non-null  object        
 6   payer_id              190792 non-null  object        
 7   endorser_original_id  190792 non-null  object        
 8   new_due_date          190792 non-null  object        
 9   created_at            190792 non-null  datetime64[ns]
 10  updated_at            190792 non-null  object        
 11  deleted_at            2700 non-null    object        
 12  participant_id        190781 non-null  object        
 13 

In [76]:
# Criando uma feature 'dias_para_vencimento' como a diferença entre 'due_date' e 'created_at'
df['dias_para_vencimento'] = (df['due_date'] - df['created_at']).dt.days


In [77]:
df.head()

,id,due_date,nfe_number,nfe_series,kind,state,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,participant_id,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at,dias_para_vencimento
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,services,canceled,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12,2024-02-09 12:56:01.418036,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,invoice,123123,São Paulo,others,NaN,24
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,goods,canceled,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07,2024-02-09 12:56:26.426682,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,BA,others,NaN,45
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,goods,active,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20,2024-02-01 03:01:47.846091,NaN,ad1b3da5-be27-4282-93df-c3bd77acac91,nfe_key,NaN,RN,NaN,NaN,28
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,goods,canceled,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29,2023-07-05 20:07:14.565196,NaN,04282b7d-ed52-4bbe-abb8-eea6cc77ba9f,nfe_key,NaN,teste,operational_error,NaN,32
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,goods,canceled,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07,2024-02-09 12:56:50.403976,NaN,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,NaN,São Paulo,others,NaN,3467


In [78]:
# Criando a variável alvo (1 = vencida, 0 = não vencida)
df['notas_vencidas'] = df['due_date'] < pd.Timestamp.now()

In [79]:
# Preenchendo valores nulos, se houver
df.fillna('Não Identificado', inplace=True)

In [80]:
df.head()

,id,due_date,nfe_number,nfe_series,kind,state,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,participant_id,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at,dias_para_vencimento,notas_vencidas
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,services,canceled,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12,2024-02-09 12:56:01.418036,Não Identificado,a11591f2-4cc7-4ff3-bb27-ad1e25361178,invoice,123123,São Paulo,others,Não Identificado,24,True
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,goods,canceled,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07,2024-02-09 12:56:26.426682,Não Identificado,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,Não Identificado,BA,others,Não Identificado,45,True
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,goods,active,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20,2024-02-01 03:01:47.846091,Não Identificado,ad1b3da5-be27-4282-93df-c3bd77acac91,nfe_key,Não Identificado,RN,Não Identificado,Não Identificado,28,True
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,goods,canceled,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29,2023-07-05 20:07:14.565196,Não Identificado,04282b7d-ed52-4bbe-abb8-eea6cc77ba9f,nfe_key,Não Identificado,teste,operational_error,Não Identificado,32,True
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,goods,canceled,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07,2024-02-09 12:56:50.403976,Não Identificado,a11591f2-4cc7-4ff3-bb27-ad1e25361178,nfe_key,Não Identificado,São Paulo,others,Não Identificado,3467,False


In [81]:
# Fazendo One-Hot Encoding para variáveis categóricas como 'state', 'kind'
df = pd.get_dummies(df, columns=['state', 'kind'], drop_first=True)

In [82]:
df.head()

,id,due_date,nfe_number,nfe_series,payer_id,endorser_original_id,new_due_date,created_at,updated_at,deleted_at,...,ballast_kind,invoice_number,payment_place,update_reason_kind,finished_at,dias_para_vencimento,notas_vencidas,state_canceled,state_finished,kind_services
0,b13369ca-e3de-451c-bfa0-2c2c06db00da,2023-08-05,523451,6,fa1370e5-9cc0-49a6-af05-f656d9657235,88c3eea1-9eab-4eaf-a9e8-1eb4a2803175,2023-08-05,2023-07-12,2024-02-09 12:56:01.418036,Não Identificado,...,invoice,123123,São Paulo,others,Não Identificado,24,True,True,False,True
1,c2f7a256-7165-4787-966e-e094344827d8,2024-03-23,000005010,001,e3d75ba8-42c9-4c99-a74b-d492d55ef4e7,964109cb-d608-444e-944c-a2eae434139a,2024-03-23,2024-02-07,2024-02-09 12:56:26.426682,Não Identificado,...,nfe_key,Não Identificado,BA,others,Não Identificado,45,True,True,False,False
2,1bedeb72-2413-43ec-95a9-c6b539ce6a99,2024-01-17,000038412,001,d072845e-b6cd-4842-ae8b-3ec786ab2c29,4cf5cf14-1f9e-4013-8246-2f8933711812,2024-01-17,2023-12-20,2024-02-01 03:01:47.846091,Não Identificado,...,nfe_key,Não Identificado,RN,Não Identificado,Não Identificado,28,True,False,False,False
3,ac2d5831-fdd6-44c5-8b07-01faa04dc451,2023-07-31,3932,1,aa4cf60b-bd06-4386-bf11-d3722f5c6aa4,1b24aaf1-a7a6-4693-867c-2ba88bf17224,2023-07-31,2023-06-29,2023-07-05 20:07:14.565196,Não Identificado,...,nfe_key,Não Identificado,teste,operational_error,Não Identificado,32,True,True,False,False
4,dacc7c5a-accb-430b-8d51-5aceb572e2c4,2033-08-05,910304872,910304872,0140a25d-625f-4431-9248-94d58419d8cd,107221ca-f7ab-464b-85c3-a19e56d4f84b,2033-08-05,2024-02-07,2024-02-09 12:56:50.403976,Não Identificado,...,nfe_key,Não Identificado,São Paulo,others,Não Identificado,3467,False,True,False,False


In [83]:
# Removendo a coluna 'payment_place'
X_without_place = df.drop(columns=['id', 'notas_vencidas', 	'nfe_number', 'invoice_number', 'nfe_series' ,'due_date', 'created_at', 'new_due_date', 'updated_at', 'deleted_at', 'finished_at', 'payment_place', 'payer_id'	,'endorser_original_id','participant_id'])


In [84]:
X_without_place.head()

,ballast_kind,update_reason_kind,dias_para_vencimento,state_canceled,state_finished,kind_services
0,invoice,others,24,True,False,True
1,nfe_key,others,45,True,False,False
2,nfe_key,Não Identificado,28,False,False,False
3,nfe_key,operational_error,32,True,False,False
4,nfe_key,others,3467,True,False,False


In [85]:
# Aplicar One-Hot Encoding em colunas categóricas como 'ballast_kind', 'update_reason_kind'
X_without_place = pd.get_dummies(X_without_place, columns=['ballast_kind', 'update_reason_kind'], drop_first=True)


In [86]:
X_without_place.head()

,dias_para_vencimento,state_canceled,state_finished,kind_services,ballast_kind_nfe_key,update_reason_kind_operational_error,update_reason_kind_others,update_reason_kind_reversal
0,24,True,False,True,False,False,True,False
1,45,True,False,False,True,False,True,False
2,28,False,False,False,True,False,False,False
3,32,True,False,False,True,True,False,False
4,3467,True,False,False,True,False,True,False


In [87]:
# Definindo a variável alvo
y = df['notas_vencidas']

In [88]:
# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train_wo, X_test_wo, y_train_wo, y_test_wo = train_test_split(X_without_place, y, test_size=0.2, random_state=42)


In [89]:
# Treinando o modelo Random Forest
rf_model_wo = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_wo.fit(X_train_wo, y_train_wo)


RandomForestClassifier(random_state=42)

In [90]:
# Fazendo previsões
y_pred_wo = rf_model_wo.predict(X_test_wo)

In [91]:
# Avaliando o modelo
accuracy_wo = accuracy_score(y_test_wo, y_pred_wo)
roc_auc_wo = roc_auc_score(y_test_wo, rf_model_wo.predict_proba(X_test_wo)[:,1])

In [92]:
print(f'Modelo SEM payment_place -> Acurácia: {accuracy_wo:.2f}, AUC-ROC: {roc_auc_wo:.2f}')

Modelo SEM payment_place -> Acurácia: 0.99, AUC-ROC: 1.00


In [94]:
# Verificando a distribuição da variável alvo (porcentagem de notas vencidas ou não)
print(df['notas_vencidas'].value_counts(normalize=True))


notas_vencidas
False    0.821497
True     0.178503
Name: proportion, dtype: float64


In [96]:
# Calcular o AUC-ROC
y_pred_proba = rf_model_wo.predict_proba(X_test_wo)[:,1]
roc_auc = roc_auc_score(y_test_wo, y_pred_proba)
print(f'AUC-ROC: {roc_auc:.2f}')

AUC-ROC: 1.00
